<a href="https://colab.research.google.com/github/anaferreira744/DE-DP-ADF/blob/main/spark/challenges/challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHALLENGE 1
##  Implement INGESTION process
- Set up path in the "lake"
  - !mkdir -p /content/lake/bronze

- Read data from API https://api.carrismetropolitana.pt/
  - Endpoints:
    - vehicles
    - lines
    - municipalities
  - Use StructFields to enforce schema

- Transformations
  - vehicles
    - create "date" extracted from "timestamp" column (format: hh24miss)

- Write data as PARQUET into the BRONZE layer (/content/lake/bronze)
  - Partition "vehicles" by "date" column
  - Paths:
    - vehicles - path: /content/lake/bronze/vehicles
    - lines - path: /content/lake/bronze/lines
    - municipalities - path: /content/lake/bronze/municipalities
  - Make sure there is only 1 single parquet created
  - Use overwrite as write mode

# Setting up PySpark

In [1]:
%pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession
         .builder
         .master('local')
         .appName('Challenge_DP_AF')
         .config('spark.ui.port', '4050')
         .getOrCreate())
sc = spark.sparkContext

In [ ]:
#Read from API
vehicles = readFromAPI(xxxxx, schema = vehicle_schema)

In [ ]:
vehicle_schema = StructType ([structField('campos', IntegerType(), True),

])